<a href="https://colab.research.google.com/github/niangmag/NLP-Assignment-3/blob/Sentiment_Analysis/Sentiment_analysis_Machine_learning_based_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sentiment analysis :Machine learning based methods

#Install Required Library


In [ ]:
!pip install transformers

#Use the Transformers pipeline
Load Transformer Sentiment Pipeline


In [ ]:
#Importing the pipeline from Transfromers.
from transformers import pipeline
sentiment_classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
import pandas as pd
import re


In [ ]:
# Load the tweets dataset (make sure 'tweets-data.csv' is in your working directory)
df = pd.read_csv("tweets-data.csv")

# Take a consistent random sample of 500 rows
df_sample = df.sample(n=500, random_state=42).reset_index(drop=True)


#Clean the tweets text

In [ ]:
def clean_text(text):
    text = str(text)
    text = re.sub(r"http\S+", "", text)          # Remove URLs
    text = re.sub(r"@\w+", "", text)             # Remove mentions
    text = re.sub(r"#", "", text)                # Remove hashtag symbol
    text = re.sub(r"[^A-Za-z\s]", "", text)      # Remove special characters/numbers
    text = text.lower().strip()                  # Convert to lowercase and strip whitespace
    return text

# Apply cleaning function
df_sample['cleaned_text'] = df_sample['Tweets'].apply(clean_text)


This function ensures consistent and clean input for the ML model.



In [ ]:
# Load the pretrained sentiment classifier pipeline from HuggingFace
sentiment_classifier = pipeline("sentiment-analysis")
def sentiment_scores_transformers(text):
    # Truncate text to 512 tokens (approx. 512 characters, conservatively)
    max_length = 512
    text = text[:max_length]

    result = sentiment_classifier(text)[0]
    return result['label'], result['score']



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


#Add 2 new columns: sentiment label and sentiment score


In [ ]:
# Apply the sentiment function to cleaned text and store results
df_sample[['sentiment_label', 'sentiment_score']] = df_sample['cleaned_text'].apply(
    lambda x: pd.Series(sentiment_scores_transformers(x))
)


In [ ]:
# Show the first 10 rows to verify output
df_sample[['Tweets', 'cleaned_text', 'sentiment_label', 'sentiment_score']].head(10)


,Tweets,cleaned_text,sentiment_label,sentiment_score
0,Le #DessinDePresse de Sanaga : ls sont morts c...,le dessindepresse de sanaga ls sont morts com...,NEGATIVE,0.981537
1,#Russia #Wagner #RussiaCivilWar https://t.co/P...,russia wagner russiacivilwar,NEGATIVE,0.962062
2,Exclusive content -https://t.co/oEiSIIB2Z1\n.\...,exclusive content \n\ncosplay japan titan tita...,NEGATIVE,0.961531
3,Auch heute geht die politische Nachricht des T...,auch heute geht die politische nachricht des t...,NEGATIVE,0.975570
4,@crazyclipsonly Same type that would take a ho...,same type that would take a homemade playstati...,NEGATIVE,0.994473
5,#SUGA_AgustD_TOUR_in_Seoul #SUGA_AgustD_TOUR #...,sugaagustdtourinseoul sugaagustdtour glastonbu...,NEGATIVE,0.970569
6,#Titan mishap: #Implosion with incredible forc...,titan mishap implosion with incredible force a...,POSITIVE,0.996252
7,#Wagner #Russia,wagner russia,POSITIVE,0.992612
8,#merri le #titanic 2 le retour https://t.co/4s...,merri le titanic le retour via,POSITIVE,0.884200
9,"Il Segretario di Stato americano #Blinken: ""no...",il segretario di stato americano blinken non c...,NEGATIVE,0.953018
